In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import numpy as np
import pandas as pd
import math
import glob
import time
import pickle

import steering
import speeding

import logging

In [2]:
PROJECT_ROOT = '../../../..'
LEARNED_DRIVER = 'snakeoil_miner/data'
DIFFICULTY = 'easy'

# COMMAND = 'steering'
COMMAND = 'speeding'
logging.basicConfig(filename='logs/{}/training-{}.log'.format(COMMAND, time.time()),level=logging.DEBUG)

NUM_EPOCHS = 100

TRAINING_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, DIFFICULTY, '*.csv']))
TRAINING_DATA = {}
for FILE in TRAINING_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    TRAINING_DATA[FILE] = DF.values

VALIDATION_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, 'validation', '*.csv']))
VALIDATION_DATA = {}
for FILE in VALIDATION_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    VALIDATION_DATA[FILE] = DF.values
    
CUDA = torch.cuda.is_available()
if CUDA:
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor

In [3]:
def save_checkpoint(state, is_best, filepath='latest_checkpoint.tar'):
    torch.save(state, 'split_checkpoints/' + COMMAND + '/' + filepath)
    if is_best:
        torch.save(state, 'split_checkpoints/' + COMMAND + '/' + 'best_checkpoint.tar')

In [4]:
def train(training_data, model, criterion, optimzier):
    loss = 0
    model.train(mode=True)
    for key in training_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        optimizer.zero_grad()
        track_sequence = training_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
        
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets)).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs),  requires_grad=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        track_loss.backward()
        optimizer.step()

        loss += track_loss.data[0]
    return loss

In [5]:
def validate(validation_data, model, criterion):
    loss = 0
    model.train(mode=False)
    for key in validation_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        track_sequence = validation_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
            
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets), volatile=True).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs), volatile=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        loss += track_loss.data[0]
    return loss

In [6]:
if COMMAND == 'steering':
    model = steering.Steering(steering.HYPERPARAMS.INPUT_SIZE,
                               steering.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               steering.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               steering.HYPERPARAMS.DROPOUT_PROB,
                               steering.HYPERPARAMS.NUM_LAYERS,
                               steering.HYPERPARAMS.TARGET_SIZE,
                               steering.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=steering.HYPERPARAMS.LEARNING_RATE)
elif COMMAND == 'speeding':
    model = speeding.Speeding(speeding.HYPERPARAMS.INPUT_SIZE,
                               speeding.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               speeding.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               speeding.HYPERPARAMS.DROPOUT_PROB,
                               speeding.HYPERPARAMS.NUM_LAYERS,
                               speeding.HYPERPARAMS.TARGET_SIZE,
                               speeding.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=speeding.HYPERPARAMS.LEARNING_RATE)
if CUDA:
    model.cuda()
        
criterion = nn.MSELoss().cuda()

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True)

min_loss = math.inf
losses = {
  'training': [],
  'validation': []
}

In [7]:
logging.info('Training %s...' % COMMAND)
print('Training %s...' % COMMAND)

for epoch in np.arange(NUM_EPOCHS):
    logging.info('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    is_best = False

    training_loss = train(TRAINING_DATA, model, criterion, optimizer)
    logging.info('--- TRAINING LOSS: %f' % training_loss)
    print('--- TRAINING LOSS: %f' % training_loss)

    validation_loss = validate(VALIDATION_DATA, model, criterion)
    logging.info('--- VALIDATION LOSS: %f' % validation_loss)
    print('--- VALIDATION LOSS: %f' % validation_loss)

    if validation_loss < min_loss:
        logging.info('--- --- best model found so far: %f' % validation_loss)
        print('--- --- best model found so far: %f' % validation_loss)
        min_loss = validation_loss
        is_best = True

    losses['training'].append(training_loss)
    losses['validation'].append(validation_loss)

    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'min_loss': min_loss,
          'optimizer' : optimizer.state_dict(),
      }, is_best)

    scheduler.step(validation_loss)
    logging.info('-------------------------------------------------------')
    print('-------------------------------------------------------')

Training speeding...
Epoch [1/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 1.326058
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.363587
--- --- best model found so far: 0.363587
-------------------------------------------------------
Epoch [2/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.475373
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.468537
-------------------------------------------------------
Epoch [3/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-r

--- VALIDATION LOSS: 0.244486
-------------------------------------------------------
Epoch [19/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.227927
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.223365
--- --- best model found so far: 0.223365
-------------------------------------------------------
Epoch [20/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.235332
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.260958
-------------------------------------------------------
Epoch 

--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.195387
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.226538
-------------------------------------------------------
Epoch [38/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.192560
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.213908
--- --- best model found so far: 0.213908
-------------------------------------------------------
Epoch [39/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.185699
--- Parsing track validation-

--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.196685
-------------------------------------------------------
Epoch [56/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.159634
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.238574
-------------------------------------------------------
Epoch [57/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.168044
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.194414
--- --- best model found so far: 0.194414
---------------------

--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.143015
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.190154
-------------------------------------------------------
Epoch [75/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.140086
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.190666
-------------------------------------------------------
Epoch [76/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.137286
--- Parsing track validation-race_32

--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.138540
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.198085
-------------------------------------------------------
Epoch [94/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.137832
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_31.csv
--- Parsing track validation-race_30.csv
--- VALIDATION LOSS: 0.198475
-------------------------------------------------------
Epoch [95/100]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.137228
-

In [8]:
# dump losses to file
with open('split_checkpoints/{}/losses.pkl'.format(COMMAND, time.time()), 'wb') as file:
    pickle.dump(losses, file)

In [9]:
# test dump worked
with open('split_checkpoints/{}/losses.pkl'.format(COMMAND, time.time()), 'rb') as file:
    hello = pickle.load(file)
hello

{'training': [1.326058343052864,
  0.4753730669617653,
  0.42906247824430466,
  0.3284917715936899,
  0.3472686782479286,
  0.31783611699938774,
  0.2965888734906912,
  0.28725307807326317,
  0.2788986284285784,
  0.2700688075274229,
  0.26362208649516106,
  0.256895350292325,
  0.2520000971853733,
  0.24524609372019768,
  0.24228928238153458,
  0.23490421287715435,
  0.23482870496809483,
  0.22985862009227276,
  0.22792719677090645,
  0.23533153161406517,
  0.2557136807590723,
  0.2275574915111065,
  0.22744071669876575,
  0.22012536693364382,
  0.21250298898667097,
  0.20706928987056017,
  0.20104627218097448,
  0.19914198759943247,
  0.1936870589852333,
  0.19087705854326487,
  0.19050700683146715,
  0.19332316052168608,
  0.19636610802263021,
  0.20455515757203102,
  0.22948798350989819,
  0.20092535018920898,
  0.19538725167512894,
  0.1925596771761775,
  0.18569886032491922,
  0.18335076980292797,
  0.17494900058954954,
  0.17289138585329056,
  0.16931734699755907,
  0.1660239296